<!-- # Διπλωματική Εργασία
## Ταξινόμηση του καρκίνου του μαστού με μεθόδους μηχανικής μάθησης
### Εξαγωγή χαρακτηριστικών με PCA

> Λάζαρος Πανιτσίδης<br />
> Τμήμα Μηχανικών Παραγωγής και Διοίκησης <br />
> Διεθνές Πανεπιστήμιο της Ελλάδος <br />
> lazarospanitsidis@outlook.com -->

# Diploma thesis
## Breast cancer classification using machine learning methods
### Feature selection with select K-Best method

> Lazaros Panitsidis<br />
> Department of Production and Management Engineering <br />
> International Hellenic University <br />
> lazarospanitsidis@outlook.com

## Contents
1. [Useful Python Libraries](#1)
1. [Data Processing](#2)
1. [Gaussian Naive Bayes](#3)
1. [Linear Discriminant Analysis](#4)
1. [Quadratic Discriminant Analysis](#5)
1. [Ridge Classifier](#6)
1. [Decision Tree Classifier](#7)
1. [Random Forest Classifier](#8)
1. [ADA Boost Classifier (Adaptive Boosting)](#9)
1. [C-Support Vector Classification](#10)
1. [Stochastic Gradient Descent Classifier](#11)
1. [eXtreme Gradient Boosting](#12)
1. [Light Gradient Boosting Machine](#13)
1. [K-Nearest Neighbors Classifier](#14)
1. [Multi-layer Perceptron Classifier](#15)
1. [Summary](#16)

<a id='1'></a>
## 1) Useful Python Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization library  
import scipy.stats as stats
import matplotlib.pyplot as plt
import time
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'
#import warnings library
import warnings
# ignore all warnings
warnings.filterwarnings('ignore')
# Any results you write to the current directory are saved as output.

# some of them are not used in this file
from sklearn.feature_selection import SelectKBest, f_classif, chi2, RFE, RFECV , mutual_info_classif
from sklearn.model_selection import train_test_split, cross_val_score , GridSearchCV , LeaveOneOut,KFold,RandomizedSearchCV
from skopt import BayesSearchCV # https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV , https://scikit-optimize.github.io/stable/auto_examples/bayesian-optimization.html
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score , make_scorer , classification_report
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline , Pipeline # https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
from sklearn.preprocessing import StandardScaler , LabelEncoder
from xgboost import XGBClassifier , plot_importance
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier , RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis , QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgbm
from sklearn.neural_network import MLPClassifier
import pygad

<a id='2'></a>
## 2) Data Processing

In [2]:
dataWISC = pd.read_csv('dataWisc.csv')
dataWISC.drop(["id", "Unnamed: 32"], axis = 1, inplace = True)

# Undersampling function
def make_undersample(_df, column):
  dfs_r = {}
  dfs_c = {}
  smaller = 1e1000
  ignore = ""
  for c in _df[column].unique():
    dfs_c[c] = _df[_df[column] == c]
    if dfs_c[c].shape[0] < smaller:
      smaller = dfs_c[c].shape[0]
      ignore = c

  for c in dfs_c:
    if c == ignore:
      continue
    dfs_r[c] = resample(dfs_c[c], 
                        replace=False, # sample without replacement
                        n_samples=smaller,
                        random_state=0)
  return pd.concat([dfs_r[c] for c in dfs_r] + [dfs_c[ignore]])

dataWISC = make_undersample(dataWISC,'diagnosis')

#Description of the dataset

#how many cases are included in the dataset
length = len(dataWISC)
#how many features are in the dataset
features = dataWISC.shape[1]-1 # - diagnosis

# Number of malignant cases
malignant = len(dataWISC[dataWISC['diagnosis']=='M'])

#Number of benign cases
benign = len(dataWISC[dataWISC['diagnosis']=='B'])

#Rate of malignant tumors over all cases
rate = (float(malignant)/(length))*100

print ("There are "+ str(len(dataWISC))+" cases in this dataset")
print ("There are {}".format(features)+" features in this dataset")
print ("There are {}".format(malignant)+" cases diagnosed as malignant tumor")
print ("There are {}".format(benign)+" cases diagnosed as benign tumor")
print ("The percentage of malignant cases is: {:.2f}%".format(rate))

There are 424 cases in this dataset
There are 30 features in this dataset
There are 212 cases diagnosed as malignant tumor
There are 212 cases diagnosed as benign tumor
The percentage of malignant cases is: 50.00%


In [3]:
y = dataWISC.diagnosis                          # M or B 
x = dataWISC.drop('diagnosis',axis = 1 )
target_names=['Benign','Malignant']
le= LabelEncoder()
le.fit(y)
y_le = le.transform(y)

In [4]:
# Select K-Best features
x_new = x[['texture_mean', 'area_mean', 'symmetry_mean', 'area_se',
       'concave points_se', 'smoothness_worst', 'concavity_worst',
       'symmetry_worst', 'fractal_dimension_worst']]
x_new.head()

,texture_mean,area_mean,symmetry_mean,area_se,concave points_se,smoothness_worst,concavity_worst,symmetry_worst,fractal_dimension_worst
49,22.30,561.0,0.1809,20.20,0.011840,0.1162,0.228200,0.2871,0.06917
285,18.40,489.0,0.1697,22.45,0.002924,0.1038,0.005579,0.2505,0.06431
495,20.21,680.9,0.1487,21.84,0.011830,0.1216,0.170000,0.2369,0.06599
391,16.84,234.3,0.1985,28.85,0.000000,0.1460,0.000000,0.2445,0.08865
187,17.19,420.3,0.1516,17.86,0.011850,0.1323,0.152100,0.2572,0.07097


In [5]:
# https://machinelearningmastery.com/loocv-for-evaluating-machine-learning-algorithms/#:~:text=Given%20the%20improved%20estimate%20of,biased%20estimates%20of%20model%20performance.
# cv = LeaveOneOut()

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
cv=KFold(n_splits=10, shuffle=True, random_state=13)

originalclass = []
predictedclass = []

def classification_report_with_accuracy_score(y_true, y_pred):
  originalclass.extend(y_true)
  predictedclass.extend(y_pred)
  #print(classification_report(y_true, y_pred, target_names=target_names)) 
  return accuracy_score(y_true, y_pred)

def print_best_params(grid_search):
    print("")
    print("Best hyperparameters : ", grid_search.best_params_)
    print("")
    print("Best estimator : ", grid_search.best_estimator_)
    print("")

<a id='3'></a>
## 3) [Gaussian Naive Bayes](<https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB>)

* Default hyperparameters

In [22]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_gnb = Pipeline([('scaler', StandardScaler()), ('gnb', GaussianNB())])
score = cross_val_score(clf_gnb, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.910     0.953     0.931       212
   Malignant      0.950     0.906     0.928       212

    accuracy                          0.929       424
   macro avg      0.930     0.929     0.929       424
weighted avg      0.930     0.929     0.929       424



* Hyperparameter tuning using Grid Search

In [23]:
param_grid = { 'gnb__var_smoothing': np.logspace(0,-10, num=100) }

grid_search = GridSearchCV(clf_gnb, param_grid=param_grid, n_jobs=-1,cv=cv,verbose=1,scoring='f1_macro')
grid_search.fit(x_new, y)

print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='mean_test_score',ascending=False).head(3)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits

Best hyperparameters :  {'gnb__var_smoothing': 0.0047508101621027985}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('gnb', GaussianNB(var_smoothing=0.0047508101621027985))])



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gnb__var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
23,0.003693,0.000774,0.002304,0.000909,0.004751,{'gnb__var_smoothing': 0.0047508101621027985},0.952851,0.905702,0.928847,0.906926,0.976177,0.976068,0.949519,0.830937,0.902778,0.97551,0.930532,0.043480,1
22,0.003098,0.000492,0.001900,0.000304,0.005995,{'gnb__var_smoothing': 0.005994842503189409},0.929624,0.905702,0.928847,0.906926,0.976177,0.976068,0.949519,0.830937,0.902778,0.97551,0.928209,0.042841,2
50,0.004055,0.001948,0.002491,0.001199,0.000009,{'gnb__var_smoothing': 8.902150854450392e-06},0.952851,0.905702,0.928847,0.906926,0.976177,0.952273,0.949519,0.830937,0.902778,0.97551,0.928152,0.041530,3


* Tuned hyperparameters

In [19]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_gnb = Pipeline(steps=[('scaler', StandardScaler()),
                ('gnb', GaussianNB(var_smoothing=0.0047508101621027985))])

score = cross_val_score(clf_gnb, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.910     0.958     0.933       212
   Malignant      0.955     0.906     0.930       212

    accuracy                          0.932       424
   macro avg      0.933     0.932     0.932       424
weighted avg      0.933     0.932     0.932       424



<a id='4'></a>
## 4) [Linear Discriminant Analysis](<https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html>)

* Default hyperparameters

In [24]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_lda = Pipeline([('scaler', StandardScaler()), ('lda', LinearDiscriminantAnalysis())])

score = cross_val_score(clf_lda, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.925     0.991     0.957       212
   Malignant      0.990     0.920     0.954       212

    accuracy                          0.955       424
   macro avg      0.957     0.955     0.955       424
weighted avg      0.957     0.955     0.955       424



* Hyperparameter tuning using Grid Search

In [25]:
param_grid = {
    'lda__solver' : ['svd','lsqr','eigen'],
    'lda__shrinkage':[None,'auto'],
    'lda__tol': [0.0001,0.001,0.01,0.1]
}

grid_search = GridSearchCV(clf_lda, param_grid=param_grid, n_jobs=-1, cv=cv,verbose=4,scoring='f1_macro')
grid_search.fit(x_new, y)

print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='mean_test_score',ascending=False).head(3)

Fitting 10 folds for each of 24 candidates, totalling 240 fits

Best hyperparameters :  {'lda__shrinkage': None, 'lda__solver': 'svd', 'lda__tol': 0.0001}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('lda', LinearDiscriminantAnalysis())])



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lda__shrinkage,param_lda__solver,param_lda__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005884,0.001442,0.004787,0.006732,None,svd,0.0001,"{'lda__shrinkage': None, 'lda__solver': 'svd',...",0.929624,0.952222,0.928847,0.930081,0.952273,0.976068,0.974437,0.926531,0.97551,1.0,0.954559,0.024576,1
7,0.009375,0.010716,0.002294,0.000779,None,lsqr,0.1,"{'lda__shrinkage': None, 'lda__solver': 'lsqr'...",0.929624,0.952222,0.928847,0.930081,0.952273,0.976068,0.974437,0.926531,0.97551,1.0,0.954559,0.024576,1
11,0.005186,0.001827,0.003095,0.001758,None,eigen,0.1,"{'lda__shrinkage': None, 'lda__solver': 'eigen...",0.929624,0.952222,0.928847,0.930081,0.952273,0.976068,0.974437,0.926531,0.97551,1.0,0.954559,0.024576,1


* Tuned hyperparameters

In [27]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_lda = Pipeline([('scaler', StandardScaler()), ('lda', LinearDiscriminantAnalysis(shrinkage=None,solver='svd',tol=0.0001))])

score = cross_val_score(clf_lda, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.925     0.991     0.957       212
   Malignant      0.990     0.920     0.954       212

    accuracy                          0.955       424
   macro avg      0.957     0.955     0.955       424
weighted avg      0.957     0.955     0.955       424



<a id='5'></a>
## 5) [Quadratic Discriminant Analysis](<https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html>)

* Default hyperparameters

In [28]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_qda = Pipeline([('scaler', StandardScaler()), ('qda', QuadraticDiscriminantAnalysis())])

score = cross_val_score(clf_qda, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.932     0.962     0.947       212
   Malignant      0.961     0.929     0.945       212

    accuracy                          0.946       424
   macro avg      0.946     0.946     0.946       424
weighted avg      0.946     0.946     0.946       424



* Hyperparameter tuning using Grid Search

In [29]:
param_grid = {
    'qda__reg_param': np.linspace(0, 1, num=10),
    'qda__tol': [0.0001,0.001,0.01]
}

grid_search = GridSearchCV(clf_qda, param_grid=param_grid, n_jobs=-1, cv=cv,verbose=4,scoring='f1_macro')
grid_search.fit(x_new, y)

print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='mean_test_score',ascending=False).head(3)

Fitting 10 folds for each of 30 candidates, totalling 300 fits

Best hyperparameters :  {'qda__reg_param': 0.0, 'qda__tol': 0.0001}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('qda', QuadraticDiscriminantAnalysis())])



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_qda__reg_param,param_qda__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004786,0.003116,0.002593,0.001353,0.0,0.0001,"{'qda__reg_param': 0.0, 'qda__tol': 0.0001}",0.929624,0.929624,0.952851,0.906522,1.0,0.976068,0.949519,0.879241,0.951389,0.97551,0.945035,0.033648,1
2,0.003590,0.001197,0.002194,0.000598,0.0,0.01,"{'qda__reg_param': 0.0, 'qda__tol': 0.01}",0.929624,0.929624,0.952851,0.906522,1.0,0.976068,0.949519,0.879241,0.951389,0.97551,0.945035,0.033648,1
1,0.003590,0.000914,0.002594,0.001277,0.0,0.001,"{'qda__reg_param': 0.0, 'qda__tol': 0.001}",0.929624,0.929624,0.952851,0.906522,1.0,0.976068,0.949519,0.879241,0.951389,0.97551,0.945035,0.033648,1


* Tuned hyperparameters

In [30]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_qda = Pipeline([('scaler', StandardScaler()), ('qda', QuadraticDiscriminantAnalysis(reg_param=0.0,tol=0.0001))])

score = cross_val_score(clf_qda, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.932     0.962     0.947       212
   Malignant      0.961     0.929     0.945       212

    accuracy                          0.946       424
   macro avg      0.946     0.946     0.946       424
weighted avg      0.946     0.946     0.946       424



<a id='6'></a>
## 6) [Ridge Classifier](<https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html#sklearn.linear_model.RidgeClassifier>)

* Default hyperparameters

In [32]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_rc = Pipeline([('scaler', StandardScaler()), ('rg', RidgeClassifier())])

score = cross_val_score(clf_rc, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.925     0.991     0.957       212
   Malignant      0.990     0.920     0.954       212

    accuracy                          0.955       424
   macro avg      0.957     0.955     0.955       424
weighted avg      0.957     0.955     0.955       424



* Hyperparameter tuning using Grid Search

In [33]:
param_grid = {
    'rg__alpha' : np.linspace(0, 1, num=10),
    'rg__fit_intercept' : [True,False],
    'rg__copy_X' : [True,False],
    'rg__max_iter' : [None],
    'rg__tol' : [0.001],
    'rg__class_weight' : [None,'balanced'],
    'rg__solver' : ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs'],
    'rg__positive' : [False]
}

grid_search = GridSearchCV(clf_rc, param_grid=param_grid, n_jobs=-1,cv=cv,verbose=1,scoring='f1_macro')
grid_search.fit(x_new, y)

print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='mean_test_score',ascending=False).head(3)

Fitting 10 folds for each of 560 candidates, totalling 5600 fits

Best hyperparameters :  {'rg__alpha': 0.0, 'rg__class_weight': None, 'rg__copy_X': True, 'rg__fit_intercept': False, 'rg__max_iter': None, 'rg__positive': False, 'rg__solver': 'svd', 'rg__tol': 0.001}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('rg',
                 RidgeClassifier(alpha=0.0, fit_intercept=False,
                                 solver='svd'))])



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rg__alpha,param_rg__class_weight,param_rg__copy_X,param_rg__fit_intercept,param_rg__max_iter,param_rg__positive,param_rg__solver,param_rg__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
233,0.005903,0.001585,0.002195,0.000748,0.444444,None,True,False,None,False,lsqr,0.001,"{'rg__alpha': 0.4444444444444444, 'rg__class_w...",0.929624,0.952222,0.928847,0.976541,0.952273,0.976068,0.974437,0.926531,0.97551,1.0,0.959205,0.023892,1
234,0.006097,0.001583,0.002605,0.000663,0.444444,None,True,False,None,False,sparse_cg,0.001,"{'rg__alpha': 0.4444444444444444, 'rg__class_w...",0.929624,0.952222,0.928847,0.976541,0.952273,0.976068,0.974437,0.926531,0.97551,1.0,0.959205,0.023892,1
25,0.005246,0.000555,0.002355,0.000676,0.0,None,False,False,None,False,sag,0.001,"{'rg__alpha': 0.0, 'rg__class_weight': None, '...",0.929624,0.952222,0.928847,0.976541,0.952273,0.976068,0.974437,0.926531,0.97551,1.0,0.959205,0.023892,1


* Tuned hyperparameters

In [35]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_rc = Pipeline([('scaler', StandardScaler()), 
                    ('rg', RidgeClassifier(alpha=0.0, fit_intercept=False,solver='svd'))])

score = cross_val_score(clf_rc, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.933     0.991     0.961       212
   Malignant      0.990     0.929     0.959       212

    accuracy                          0.960       424
   macro avg      0.962     0.960     0.960       424
weighted avg      0.962     0.960     0.960       424



<a id='7'></a>
## 7) [Decision Tree Classifier](<https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html>)

* Default hyperparameters

In [41]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_tree = Pipeline([('scaler', StandardScaler()), ('tree', DecisionTreeClassifier(random_state=13))])

score = cross_val_score(clf_tree, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.938     0.920     0.929       212
   Malignant      0.921     0.939     0.930       212

    accuracy                          0.929       424
   macro avg      0.929     0.929     0.929       424
weighted avg      0.929     0.929     0.929       424



* Hyperparameter tuning using Grid Search

In [37]:
param_grid = {
    'tree__criterion' :['gini','entropy'],
    'tree__splitter' : ['best','random'],
    'tree__max_depth': [2,6,10,None],
    'tree__min_samples_split': list(range(2, 4)),
    'tree__min_samples_leaf': [3,5],
    'tree__min_weight_fraction_leaf' : [0.0],
    'tree__max_features': [None, 'sqrt', 'log2'],
    'tree__max_leaf_nodes' : [None,10,50],
    'tree__min_impurity_decrease' : [0.0],
    'tree__class_weight' : [None,'balanced'],
    'tree__ccp_alpha' : [0.0],
    'tree__random_state' : [13]
}

grid_search = GridSearchCV(clf_tree, param_grid=param_grid, n_jobs=-1,cv=cv,verbose=1,scoring='f1_macro')
grid_search.fit(x_new, y)

print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='mean_test_score',ascending=False).head(3)

Fitting 10 folds for each of 1152 candidates, totalling 11520 fits

Best hyperparameters :  {'tree__ccp_alpha': 0.0, 'tree__class_weight': 'balanced', 'tree__criterion': 'gini', 'tree__max_depth': 6, 'tree__max_features': 'sqrt', 'tree__max_leaf_nodes': 10, 'tree__min_impurity_decrease': 0.0, 'tree__min_samples_leaf': 3, 'tree__min_samples_split': 2, 'tree__min_weight_fraction_leaf': 0.0, 'tree__random_state': 13, 'tree__splitter': 'best'}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('tree',
                 DecisionTreeClassifier(class_weight='balanced', max_depth=6,
                                        max_features='sqrt', max_leaf_nodes=10,
                                        min_samples_leaf=3, random_state=13))])



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_tree__ccp_alpha,param_tree__class_weight,param_tree__criterion,param_tree__max_depth,param_tree__max_features,param_tree__max_leaf_nodes,param_tree__min_impurity_decrease,param_tree__min_samples_leaf,param_tree__min_samples_split,param_tree__min_weight_fraction_leaf,param_tree__random_state,param_tree__splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
754,0.003790,0.000598,0.001895,0.000537,0.0,balanced,gini,10,sqrt,10,0.0,3,3,0.0,13,best,"{'tree__ccp_alpha': 0.0, 'tree__class_weight':...",0.976282,0.953261,0.929624,0.928847,0.976177,0.952273,0.974437,0.854167,0.951389,0.925134,0.942159,0.0347,1
824,0.004189,0.002034,0.001895,0.000537,0.0,balanced,gini,None,sqrt,10,0.0,3,2,0.0,13,best,"{'tree__ccp_alpha': 0.0, 'tree__class_weight':...",0.976282,0.953261,0.929624,0.928847,0.976177,0.952273,0.974437,0.854167,0.951389,0.925134,0.942159,0.0347,1
752,0.003690,0.000639,0.001895,0.000299,0.0,balanced,gini,10,sqrt,10,0.0,3,2,0.0,13,best,"{'tree__ccp_alpha': 0.0, 'tree__class_weight':...",0.976282,0.953261,0.929624,0.928847,0.976177,0.952273,0.974437,0.854167,0.951389,0.925134,0.942159,0.0347,1


* Tuned hyperparameters

In [38]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_tree = Pipeline([('scaler', StandardScaler()),
                    ('tree', DecisionTreeClassifier(class_weight='balanced', max_depth=6,
                                        max_features='sqrt', max_leaf_nodes=10,
                                        min_samples_leaf=3, random_state=13))])

score = cross_val_score(clf_tree, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.943     0.943     0.943       212
   Malignant      0.943     0.943     0.943       212

    accuracy                          0.943       424
   macro avg      0.943     0.943     0.943       424
weighted avg      0.943     0.943     0.943       424



<a id='8'></a>
## 8) [Random Forest Classifier](<https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html>)

* Default hyperparameters

In [86]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_rf = Pipeline([('scaler', StandardScaler()), ('rf', RandomForestClassifier(random_state=13))])
                       
score = cross_val_score(clf_rf, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.976     0.962     0.969       212
   Malignant      0.963     0.976     0.970       212

    accuracy                          0.969       424
   macro avg      0.969     0.969     0.969       424
weighted avg      0.969     0.969     0.969       424



* Hyperparameter tuning using Grid Search

In [87]:
param_grid = {
    'rf__bootstrap': [True,False],
    'rf__max_depth': [5, 10 , None],
    'rf__n_estimators' : [10,50,100,200,500],
    'rf__max_features': [None, 'sqrt', 'log2'],
    'rf__max_leaf_nodes' : [None,5,10],
    'rf__min_samples_leaf': [1,3,5],
    'rf__min_samples_split': list(range(2, 6)),
    'rf__criterion' :['entropy','gini'],
}

grid_search = GridSearchCV(clf_rf, param_grid=param_grid, n_jobs=-1,cv=cv,verbose=1,scoring='f1_macro')
grid_search.fit(x_new, y)

print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='mean_test_score',ascending=False).head(3)

Fitting 10 folds for each of 6480 candidates, totalling 64800 fits

Best hyperparameters :  {'rf__bootstrap': True, 'rf__criterion': 'gini', 'rf__max_depth': 10, 'rf__max_features': 'sqrt', 'rf__max_leaf_nodes': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 100, 'rf__random_state': 13}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('rf',
                 RandomForestClassifier(max_depth=10, max_features='sqrt',
                                        min_samples_split=5,
                                        random_state=13))])



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rf__bootstrap,param_rf__criterion,param_rf__max_depth,param_rf__max_features,param_rf__max_leaf_nodes,param_rf__min_samples_leaf,param_rf__min_samples_split,param_rf__n_estimators,param_rf__random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
2357,0.133643,0.014739,0.010671,0.001265,True,gini,10,sqrt,None,1,5,100,13,"{'rf__bootstrap': True, 'rf__criterion': 'gini...",1.0,0.953261,0.952851,0.976541,0.952381,0.952273,0.974437,0.951945,1.0,1.0,0.971369,0.020661,1
2897,0.131249,0.005592,0.010571,0.000662,True,gini,None,sqrt,None,1,5,100,13,"{'rf__bootstrap': True, 'rf__criterion': 'gini...",1.0,0.953261,0.952851,0.976541,0.952381,0.952273,0.974437,0.951945,1.0,1.0,0.971369,0.020661,1
3077,0.149500,0.044630,0.012467,0.007533,True,gini,None,log2,None,1,5,100,13,"{'rf__bootstrap': True, 'rf__criterion': 'gini...",1.0,0.953261,0.952851,0.976541,0.952381,0.952273,0.974437,0.951945,1.0,1.0,0.971369,0.020661,1


* Tuned hyperparameters

In [88]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_rf = Pipeline(steps=[('scaler', StandardScaler()),
                ('rf',RandomForestClassifier(max_depth=10, max_features='sqrt',
                                        min_samples_split=5,
                                        random_state=13))])
                       
score = cross_val_score(clf_rf, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.976     0.967     0.972       212
   Malignant      0.967     0.976     0.972       212

    accuracy                          0.972       424
   macro avg      0.972     0.972     0.972       424
weighted avg      0.972     0.972     0.972       424



<a id='9'></a>
## 9) [ADA Boost Classifier (Adaptive Boosting)](<https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#:~:text=An%20AdaBoost%20%5B1%5D%20classifier%20is,focus%20more%20on%20difficult%20cases.>)

* Default hyperparameters

In [42]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_adaboost = Pipeline([('scaler', StandardScaler()), ('adab', AdaBoostClassifier(random_state=13))])

score = cross_val_score(clf_adaboost, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.958     0.972     0.965       212
   Malignant      0.971     0.958     0.964       212

    accuracy                          0.965       424
   macro avg      0.965     0.965     0.965       424
weighted avg      0.965     0.965     0.965       424



* Hyperparameter tuning using Grid Search

In [43]:
param_grid = {
    'adab__base_estimator' : [DecisionTreeClassifier(class_weight='balanced', max_depth=6,max_features='sqrt', max_leaf_nodes=10,min_samples_leaf=3, random_state=13)],
    'adab__n_estimators' : [10,50,100,500],
    'adab__learning_rate' : np.power(10, np.arange(-3, 1, dtype=float)),
    'adab__algorithm' : ['SAMME', 'SAMME.R'],
    'adab__random_state' : [13],
}

grid_search = GridSearchCV(clf_adaboost, param_grid=param_grid, n_jobs=-1,cv=cv,verbose=1,scoring='f1_macro')
grid_search.fit(x_new, y)

print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='mean_test_score',ascending=False).head(3)

Fitting 10 folds for each of 32 candidates, totalling 320 fits

Best hyperparameters :  {'adab__algorithm': 'SAMME', 'adab__base_estimator': DecisionTreeClassifier(class_weight='balanced', max_depth=6,
                       max_features='sqrt', max_leaf_nodes=10,
                       min_samples_leaf=3, random_state=13), 'adab__learning_rate': 1.0, 'adab__n_estimators': 500, 'adab__random_state': 13}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('adab',
                 AdaBoostClassifier(algorithm='SAMME',
                                    base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                                          max_depth=6,
                                                                          max_features='sqrt',
                                                                          max_leaf_nodes=10,
                                                                          

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_adab__algorithm,param_adab__base_estimator,param_adab__learning_rate,param_adab__n_estimators,param_adab__random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
15,1.087824,0.130720,0.044499,0.004348,SAMME,DecisionTreeClassifier(class_weight='balanced'...,1.0,500,13,"{'adab__algorithm': 'SAMME', 'adab__base_estim...",1.000000,0.952851,0.952851,0.976541,0.976177,0.952273,0.974437,0.951945,1.0,0.97551,0.971258,0.017730,1
14,0.229124,0.037196,0.010239,0.001409,SAMME,DecisionTreeClassifier(class_weight='balanced'...,1.0,100,13,"{'adab__algorithm': 'SAMME', 'adab__base_estim...",0.975910,0.929624,0.952851,0.976541,0.976177,0.952273,0.974437,0.951945,1.0,0.97551,0.966527,0.018735,2
13,0.128910,0.043983,0.009737,0.004492,SAMME,DecisionTreeClassifier(class_weight='balanced'...,1.0,50,13,"{'adab__algorithm': 'SAMME', 'adab__base_estim...",0.976282,0.929624,0.952851,0.976541,0.976177,0.952273,0.974437,0.928205,1.0,0.97551,0.964190,0.021722,3


* Tuned hyperparameters

In [44]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_adaboost = Pipeline(steps=[('scaler', StandardScaler()),
                ('adab',AdaBoostClassifier(algorithm='SAMME',
                                    base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                                          max_depth=6,
                                                                          max_features='sqrt',
                                                                          max_leaf_nodes=10,
                                                                          min_samples_leaf=3,
                                                                          random_state=13),
                                    n_estimators=500, random_state=13))])

score = cross_val_score(clf_adaboost, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.967     0.976     0.972       212
   Malignant      0.976     0.967     0.972       212

    accuracy                          0.972       424
   macro avg      0.972     0.972     0.972       424
weighted avg      0.972     0.972     0.972       424



<a id='10'></a>
## 10) [C-Support Vector Classification](<https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html>)

* Default hyperparameters

In [52]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_svc = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])

score = cross_val_score(clf_svc, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.963     0.981     0.972       212
   Malignant      0.981     0.962     0.971       212

    accuracy                          0.972       424
   macro avg      0.972     0.972     0.972       424
weighted avg      0.972     0.972     0.972       424



* Hyperparameter tuning using Grid Search

In [53]:
param_grid = [
    {
        'svc__kernel': ['rbf'], 
        'svc__gamma': [1e-2, 1e-3, 1e-4,'auto','scale'], 
        'svc__C': [1, 10, 100, 1000],
        'svc__decision_function_shape': ['ovo', 'ovr'],
        'svc__random_state' : [13]
    },
    {
        'svc__kernel': ['linear'], 
        'svc__C': [1, 10, 100, 1000],
        'svc__decision_function_shape': ['ovo', 'ovr'],
        'svc__random_state' : [13]
    },
]

grid_search = GridSearchCV(clf_svc, param_grid=param_grid, n_jobs=-1,cv=cv,verbose=1,scoring='f1_macro')
grid_search.fit(x_new, y)

print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='mean_test_score',ascending=False).head(3)

Fitting 10 folds for each of 48 candidates, totalling 480 fits

Best hyperparameters :  {'svc__C': 100, 'svc__decision_function_shape': 'ovo', 'svc__gamma': 0.001, 'svc__kernel': 'rbf', 'svc__random_state': 13}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('svc',
                 SVC(C=100, decision_function_shape='ovo', gamma=0.001,
                     random_state=13))])



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__decision_function_shape,param_svc__gamma,param_svc__kernel,param_svc__random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
26,0.004987,0.000631,0.003790,0.004411,100,ovr,0.001,rbf,13,"{'svc__C': 100, 'svc__decision_function_shape'...",0.952851,0.976282,0.952851,0.976541,0.976177,0.976068,0.974437,0.928205,1.0,1.0,0.971341,0.020722,1
37,0.004787,0.000746,0.002094,0.000537,1000,ovr,0.0001,rbf,13,"{'svc__C': 1000, 'svc__decision_function_shape...",0.952851,0.976282,0.952851,0.976541,0.976177,0.976068,0.974437,0.928205,1.0,1.0,0.971341,0.020722,1
32,0.005286,0.000779,0.002294,0.000457,1000,ovo,0.0001,rbf,13,"{'svc__C': 1000, 'svc__decision_function_shape...",0.952851,0.976282,0.952851,0.976541,0.976177,0.976068,0.974437,0.928205,1.0,1.0,0.971341,0.020722,1


* Tuned hyperparameters

In [54]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_svc = Pipeline(steps=[('scaler', StandardScaler()),
                ('svc',SVC(C=100, decision_function_shape='ovo', gamma=0.001,
                     random_state=13))])

score = cross_val_score(clf_svc, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.963     0.981     0.972       212
   Malignant      0.981     0.962     0.971       212

    accuracy                          0.972       424
   macro avg      0.972     0.972     0.972       424
weighted avg      0.972     0.972     0.972       424



<a id='11'></a>
## 11) [Stochastic Gradient Descent Classifier](<https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html>)

* Default hyperparameters

In [55]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_sgd = Pipeline([('scaler', StandardScaler()), ('sgd', SGDClassifier(random_state=13))])

score = cross_val_score(clf_sgd, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.962     0.958     0.960       212
   Malignant      0.958     0.962     0.960       212

    accuracy                          0.960       424
   macro avg      0.960     0.960     0.960       424
weighted avg      0.960     0.960     0.960       424



* Hyperparameter tuning using Grid Search

In [56]:
param_grid = {
    'sgd__average': [True, False],
    'sgd__l1_ratio': np.linspace(0, 1, num=10),
    'sgd__alpha': np.power(10, np.arange(-2, 1, dtype=float)),
    'sgd__random_state' : [13]
}

grid_search = GridSearchCV(clf_sgd, param_grid=param_grid, n_jobs=-1, cv=cv,verbose=4,scoring='f1_macro')
grid_search.fit(x_new, y)

print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='mean_test_score',ascending=False).head(3)

Fitting 10 folds for each of 60 candidates, totalling 600 fits

Best hyperparameters :  {'sgd__alpha': 0.01, 'sgd__average': False, 'sgd__l1_ratio': 0.0, 'sgd__random_state': 13}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('sgd',
                 SGDClassifier(alpha=0.01, l1_ratio=0.0, random_state=13))])



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_sgd__alpha,param_sgd__average,param_sgd__l1_ratio,param_sgd__random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
17,0.004488,0.001561,0.001995,0.000446,0.01,False,0.777778,13,"{'sgd__alpha': 0.01, 'sgd__average': False, 's...",0.976282,0.976282,0.952851,0.976541,0.976177,0.976068,0.974437,0.928205,1.0,1.0,0.973684,0.019803,1
11,0.006483,0.003996,0.002793,0.001396,0.01,False,0.111111,13,"{'sgd__alpha': 0.01, 'sgd__average': False, 's...",0.976282,0.976282,0.952851,0.976541,0.976177,0.976068,0.974437,0.928205,1.0,1.0,0.973684,0.019803,1
19,0.003890,0.001133,0.001995,0.000446,0.01,False,1.0,13,"{'sgd__alpha': 0.01, 'sgd__average': False, 's...",0.976282,0.976282,0.952851,0.976541,0.976177,0.976068,0.974437,0.928205,1.0,1.0,0.973684,0.019803,1


* Tuned hyperparameters

In [58]:
originalclass = []
predictedclass = []

# Cross validate
clf_sgd = Pipeline(steps=[('scaler', StandardScaler()),
                ('sgd',SGDClassifier(alpha=0.01, l1_ratio=0.0, random_state=13))])

score = cross_val_score(clf_sgd, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.967     0.981     0.974       212
   Malignant      0.981     0.967     0.974       212

    accuracy                          0.974       424
   macro avg      0.974     0.974     0.974       424
weighted avg      0.974     0.974     0.974       424



<a id='12'></a>
## 12) [eXtreme Gradient Boosting](<https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters>)

* Default hyperparameters

In [6]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_xgb = Pipeline([('scaler', StandardScaler()), ('xgb', XGBClassifier(random_state=13))])

score = cross_val_score(clf_xgb, x_new, y_le, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.958     0.958     0.958       212
   Malignant      0.958     0.958     0.958       212

    accuracy                          0.958       424
   macro avg      0.958     0.958     0.958       424
weighted avg      0.958     0.958     0.958       424



* Hyperparameter tuning using Grid Search

In [62]:
# https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning/notebook
# https://www.cs.cornell.edu/courses/cs4780/2018sp/lectures/lecturenote19.html
# https://medium.com/data-design/xgboost-hi-im-gamma-what-can-i-do-for-you-and-the-tuning-of-regularization-a42ea17e6ab6

param_grid = {
        'xgb__booster' : ['gbtree'],
        'xgb__validate_parameters' : [True],
        'xgb__learning_rate' : [0.05,0.1,0.3,0.5,1],
        'xgb__gamma' : [0,0.01,0.1,0.5,1],
        'xgb__max_depth' : [2,6,10],
        'xgb__min_child_weight' : [1,3,5],
        'xgb__max_delta_step' : [0,2,4],
        'xgb__subsample' : [0.5],
        'xgb__colsample_bylevel' : [1],
        'xgb__colsample_bynode' : [1],
        'xgb__colsample_bytree' : [1],
        'xgb__reg_lambda' : [0,1],
        'xgb__reg_alpha' : [0],
        'xgb__tree_method' : ['exact'],
        'xgb__scale_pos_weight' : [1],
        'xgb__objective' : ['binary:logistic'], # 'multi:softmax' -> same scores as 'binary:logistic'
        #'num_class' : [2],
        'xgb__n_estimators' : [50,100,200,500],
        'xgb__random_state' : [13]
    }

grid_search = GridSearchCV(clf_xgb, param_grid=param_grid, n_jobs=-1,cv=cv,verbose=1,scoring='f1_macro')
grid_search.fit(x_new, y_le)

print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='mean_test_score',ascending=False).head(3)

Fitting 10 folds for each of 5400 candidates, totalling 54000 fits

Best hyperparameters :  {'xgb__booster': 'gbtree', 'xgb__colsample_bylevel': 1, 'xgb__colsample_bynode': 1, 'xgb__colsample_bytree': 1, 'xgb__gamma': 0.5, 'xgb__learning_rate': 1, 'xgb__max_delta_step': 2, 'xgb__max_depth': 2, 'xgb__min_child_weight': 1, 'xgb__n_estimators': 100, 'xgb__objective': 'binary:logistic', 'xgb__random_state': 13, 'xgb__reg_alpha': 0, 'xgb__reg_lambda': 1, 'xgb__scale_pos_weight': 1, 'xgb__subsample': 0.5, 'xgb__tree_method': 'exact', 'xgb__validate_parameters': True}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('xgb',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               gamma

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_xgb__booster,param_xgb__colsample_bylevel,param_xgb__colsample_bynode,param_xgb__colsample_bytree,param_xgb__gamma,param_xgb__learning_rate,param_xgb__max_delta_step,param_xgb__max_depth,param_xgb__min_child_weight,param_xgb__n_estimators,param_xgb__objective,param_xgb__random_state,param_xgb__reg_alpha,param_xgb__reg_lambda,param_xgb__scale_pos_weight,param_xgb__subsample,param_xgb__tree_method,param_xgb__validate_parameters,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
4179,0.093504,0.035124,0.004140,0.003323,gbtree,1,1,1,0.5,1,2,2,1,100,binary:logistic,13,0,1,1,0.5,exact,True,"{'xgb__booster': 'gbtree', 'xgb__colsample_byl...",1.000000,0.976282,0.928847,0.952851,0.976177,0.952273,1.000000,0.951945,1.0,0.927545,0.966592,0.026630,1
2498,0.078293,0.002951,0.002992,0.000631,gbtree,1,1,1,0.1,0.1,2,10,1,100,binary:logistic,13,0,0,1,0.5,exact,True,"{'xgb__booster': 'gbtree', 'xgb__colsample_byl...",0.976282,0.952851,0.928847,0.952851,0.976177,0.952273,0.974437,0.951945,1.0,1.000000,0.966566,0.021683,2
2402,0.102729,0.026714,0.004088,0.001753,gbtree,1,1,1,0.1,0.1,0,6,1,100,binary:logistic,13,0,0,1,0.5,exact,True,"{'xgb__booster': 'gbtree', 'xgb__colsample_byl...",0.976282,0.952851,0.928847,0.952851,0.976177,0.952273,0.974437,0.951945,1.0,1.000000,0.966566,0.021683,2


* Tuned hyperparameters

In [7]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_xgb = Pipeline(steps=[('scaler', StandardScaler()),
                ('xgb',XGBClassifier(booster='gbtree',gamma=0.5,learning_rate=1,max_delta_step=2,max_depth=2,min_child_weight=1,
                                    n_estimators=100,objective='binary:logistic',reg_alpha=0,reg_lambda=1,scale_pos_weight=1,subsample=0.5,
                                    tree_method='exact',validate_parameters=True,random_state=13))])

score = cross_val_score(clf_xgb, x_new, y_le, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.958     0.976     0.967       212
   Malignant      0.976     0.958     0.967       212

    accuracy                          0.967       424
   macro avg      0.967     0.967     0.967       424
weighted avg      0.967     0.967     0.967       424



<a id='13'></a>
## 13) [Light Gradient Boosting Machine](<https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html>)

* Default hyperparameters

In [67]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_lgbm = Pipeline([('scaler', StandardScaler()), ('lgbm', lgbm.LGBMClassifier(random_state=13))])

score = cross_val_score(clf_lgbm, x_new, y_le, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.953     0.967     0.960       212
   Malignant      0.967     0.953     0.960       212

    accuracy                          0.960       424
   macro avg      0.960     0.960     0.960       424
weighted avg      0.960     0.960     0.960       424



* Hyperparameter tuning using Grid Search

In [79]:
# https://neptune.ai/blog/lightgbm-parameters-guide
# https://www.youtube.com/watch?v=5CWwwtEM2TA&ab_channel=PyData & https://github.com/MSusik/newgradientboosting/blob/master/pydata.pdf

param_grid = {
        'lgbm__boosting_type' : ['gbdt','dart'],
        'lgbm__num_leaves' : [10,20,30,40,50],
        'lgbm__max_depth' : [3,6,9,-1],
        'lgbm__learning_rate' : [0.05,0.1,0.3,0.5,1],
        'lgbm__n_estimators' : [50,100,200,500],
        'lgbm__objective' : ['binary'],
        'lgbm__min_child_samples' : [10,20,30],
        'lgbm__subsample' : [0.5],
        'lgbm__reg_lambda' : [0,1],
        'lgbm__reg_alpha' : [0],
        'lgbm__subsample' : [0.5],
        'lgbm__colsample_bytree' : [1],
        'lgbm__scale_pos_weight' : [1],
        'lgbm__random_state' : [13]
    }

grid_search = GridSearchCV(clf_lgbm, param_grid=param_grid, n_jobs=-1,cv=cv,verbose=1,scoring='f1_macro')
grid_search.fit(x_new, y_le)

print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='mean_test_score',ascending=False).head(3)

Fitting 10 folds for each of 4800 candidates, totalling 48000 fits

Best hyperparameters :  {'lgbm__boosting_type': 'dart', 'lgbm__colsample_bytree': 1, 'lgbm__learning_rate': 1, 'lgbm__max_depth': 6, 'lgbm__min_child_samples': 30, 'lgbm__n_estimators': 100, 'lgbm__num_leaves': 10, 'lgbm__objective': 'binary', 'lgbm__random_state': 13, 'lgbm__reg_alpha': 0, 'lgbm__reg_lambda': 0, 'lgbm__scale_pos_weight': 1, 'lgbm__subsample': 0.5}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('lgbm',
                 LGBMClassifier(boosting_type='dart', colsample_bytree=1,
                                learning_rate=1, max_depth=6,
                                min_child_samples=30, num_leaves=10,
                                objective='binary', random_state=13,
                                reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
                                subsample=0.5))])



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lgbm__boosting_type,param_lgbm__colsample_bytree,param_lgbm__learning_rate,param_lgbm__max_depth,param_lgbm__min_child_samples,param_lgbm__n_estimators,param_lgbm__num_leaves,param_lgbm__objective,param_lgbm__random_state,param_lgbm__reg_alpha,param_lgbm__reg_lambda,param_lgbm__scale_pos_weight,param_lgbm__subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
4530,0.029023,0.001574,0.002393,0.000662,dart,1,1,6,30,100,10,binary,13,0,0,1,0.5,"{'lgbm__boosting_type': 'dart', 'lgbm__colsamp...",0.976282,0.952851,0.928847,0.976541,0.976177,0.976068,1.000000,0.927545,1.0,0.97551,0.968982,0.024015,1
2221,0.042439,0.007135,0.003391,0.003222,gbdt,1,1,9,20,200,10,binary,13,0,1,1,0.5,"{'lgbm__boosting_type': 'gbdt', 'lgbm__colsamp...",1.000000,0.928847,0.952851,0.976541,0.976177,0.952273,0.974437,0.951945,1.0,0.97551,0.968858,0.021321,2
2231,0.092107,0.014323,0.002793,0.000399,gbdt,1,1,9,20,500,10,binary,13,0,1,1,0.5,"{'lgbm__boosting_type': 'gbdt', 'lgbm__colsamp...",1.000000,0.928847,0.952851,0.976541,0.976177,0.952273,0.974437,0.951945,1.0,0.97551,0.968858,0.021321,2


* Tuned hyperparameters

In [81]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_lgbm = Pipeline(steps=[('scaler', StandardScaler()),
                ('lgbm',lgbm.LGBMClassifier(boosting_type='dart', colsample_bytree=1,
                                learning_rate=1, max_depth=6,
                                min_child_samples=30, num_leaves=10,
                                objective='binary', random_state=13,
                                reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
                                subsample=0.5))])

score = cross_val_score(clf_lgbm, x_new, y_le, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.954     0.986     0.970       212
   Malignant      0.985     0.953     0.969       212

    accuracy                          0.969       424
   macro avg      0.970     0.969     0.969       424
weighted avg      0.970     0.969     0.969       424



<a id='14'></a>
## 14) [K-Nearest Neighbors Classifier](<https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html>)

* Default hyperparameters

In [69]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_knn = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier())])

score = cross_val_score(clf_knn, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.945     0.976     0.961       212
   Malignant      0.976     0.943     0.959       212

    accuracy                          0.960       424
   macro avg      0.960     0.960     0.960       424
weighted avg      0.960     0.960     0.960       424



* Hyperparameter tuning using Grid Search

In [70]:
param_grid = {
    'knn__n_neighbors': list(range(2,10)),
    'knn__weights': ['uniform','distance'],
    'knn__algorithm' : ['ball_tree', 'kd_tree', 'brute'],
    'knn__leaf_size': [10,20,30,40,50],
    'knn__p': [1,2],
    'knn__metric': ['minkowski','manhattan','chebyshev']
}

grid_search = GridSearchCV(clf_knn, param_grid=param_grid, n_jobs=-1,cv=cv,verbose=1,scoring='f1_macro')
grid_search.fit(x_new, y)

print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='mean_test_score',ascending=False).head(3)

Fitting 10 folds for each of 1440 candidates, totalling 14400 fits

Best hyperparameters :  {'knn__algorithm': 'ball_tree', 'knn__leaf_size': 10, 'knn__metric': 'minkowski', 'knn__n_neighbors': 4, 'knn__p': 1, 'knn__weights': 'distance'}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('knn',
                 KNeighborsClassifier(algorithm='ball_tree', leaf_size=10,
                                      n_neighbors=4, p=1,
                                      weights='distance'))])



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_knn__algorithm,param_knn__leaf_size,param_knn__metric,param_knn__n_neighbors,param_knn__p,param_knn__weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
521,0.003350,0.000633,0.002350,0.000444,kd_tree,10,manhattan,4,1,distance,"{'knn__algorithm': 'kd_tree', 'knn__leaf_size'...",0.952851,0.952851,0.952851,0.953261,0.976177,0.928531,0.974437,0.927545,1.0,1.0,0.96185,0.024245,1
969,0.003740,0.001075,0.003791,0.001246,brute,10,minkowski,4,1,distance,"{'knn__algorithm': 'brute', 'knn__leaf_size': ...",0.952851,0.952851,0.952851,0.953261,0.976177,0.928531,0.974437,0.927545,1.0,1.0,0.96185,0.024245,1
297,0.002897,0.000301,0.002952,0.001189,ball_tree,40,minkowski,4,1,distance,"{'knn__algorithm': 'ball_tree', 'knn__leaf_siz...",0.952851,0.952851,0.952851,0.953261,0.976177,0.928531,0.974437,0.927545,1.0,1.0,0.96185,0.024245,1


* Tuned hyperparameters

In [71]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_knn = Pipeline(steps=[('scaler', StandardScaler()),
                ('knn',KNeighborsClassifier(algorithm='ball_tree', leaf_size=10,
                                      n_neighbors=4, p=1,
                                      weights='distance'))])

score = cross_val_score(clf_knn, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.954     0.972     0.963       212
   Malignant      0.971     0.953     0.962       212

    accuracy                          0.962       424
   macro avg      0.962     0.962     0.962       424
weighted avg      0.962     0.962     0.962       424



<a id='15'></a>
## 15) [Multi-layer Perceptron Classifier](<https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html>)

* Default hyperparameters

In [75]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_mlp =  Pipeline([('scaler', StandardScaler()), ('mlp', MLPClassifier(random_state=13))])

score = cross_val_score(clf_mlp, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.971     0.962     0.967       212
   Malignant      0.963     0.972     0.967       212

    accuracy                          0.967       424
   macro avg      0.967     0.967     0.967       424
weighted avg      0.967     0.967     0.967       424



* Hyperparameter tuning using Grid Search

In [76]:
# https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
param_grid = {
    'mlp__hidden_layer_sizes' : [(18,39,)],
    'mlp__activation' : ['tanh','relu'],
    'mlp__solver' : ['sgd','adam'],
    'mlp__alpha' : [0.01,0,2],
    'mlp__batch_size' : [40,80,'auto'],
    'mlp__learning_rate' : ['invscaling','adaptive'],
    'mlp__learning_rate_init' : np.power(10, np.arange(-3, 0, dtype=float)),
    'mlp__power_t' : [0.5],
    'mlp__max_iter' : [50,100,200,500],
    'mlp__shuffle' : [True],
    'mlp__random_state' : [13]
}

grid_search = GridSearchCV(clf_mlp, param_grid=param_grid, n_jobs=-1,cv=cv,verbose=1,scoring='f1_macro')
grid_search.fit(x_new, y)

print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='rank_test_score').head(3)

Fitting 10 folds for each of 864 candidates, totalling 8640 fits

Best hyperparameters :  {'mlp__activation': 'tanh', 'mlp__alpha': 0, 'mlp__batch_size': 40, 'mlp__hidden_layer_sizes': (18, 39), 'mlp__learning_rate': 'invscaling', 'mlp__learning_rate_init': 0.1, 'mlp__max_iter': 50, 'mlp__power_t': 0.5, 'mlp__random_state': 13, 'mlp__shuffle': True, 'mlp__solver': 'sgd'}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 MLPClassifier(activation='tanh', alpha=0, batch_size=40,
                               hidden_layer_sizes=(18, 39),
                               learning_rate='invscaling',
                               learning_rate_init=0.1, max_iter=50,
                               random_state=13, solver='sgd'))])



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_mlp__activation,param_mlp__alpha,param_mlp__batch_size,param_mlp__hidden_layer_sizes,param_mlp__learning_rate,param_mlp__learning_rate_init,param_mlp__max_iter,param_mlp__power_t,param_mlp__random_state,param_mlp__shuffle,param_mlp__solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
160,0.232841,0.031207,0.002294,6.384974e-04,tanh,0,40,"(18, 39)",invscaling,0.1,50,0.5,13,True,sgd,"{'mlp__activation': 'tanh', 'mlp__alpha': 0, '...",1.000000,0.976282,0.952851,0.952851,0.976177,0.976068,0.974437,0.951945,1.0,1.0,0.976061,0.018387,1
382,0.350363,0.029623,0.001995,3.234067e-07,tanh,2,80,"(18, 39)",adaptive,0.1,500,0.5,13,True,sgd,"{'mlp__activation': 'tanh', 'mlp__alpha': 2, '...",0.976282,0.976282,0.952851,0.976541,0.976177,0.976068,0.974437,0.951945,1.0,1.0,0.976058,0.015065,2
814,0.347571,0.038948,0.001795,3.989937e-04,relu,2,80,"(18, 39)",adaptive,0.1,500,0.5,13,True,sgd,"{'mlp__activation': 'relu', 'mlp__alpha': 2, '...",0.976282,0.976282,0.952851,0.976541,0.976177,0.976068,0.974437,0.951945,1.0,1.0,0.976058,0.015065,2


* Tuned hyperparameters

In [77]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_mlp =  Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',MLPClassifier(activation='tanh', alpha=0, batch_size=40,
                               hidden_layer_sizes=(18, 39),
                               learning_rate='invscaling',
                               learning_rate_init=0.1, max_iter=50,
                               random_state=13, solver='sgd'))])

score = cross_val_score(clf_mlp, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.972     0.981     0.977       212
   Malignant      0.981     0.972     0.976       212

    accuracy                          0.976       424
   macro avg      0.976     0.976     0.976       424
weighted avg      0.976     0.976     0.976       424



* Tried a larger range of hyperparameters for testing at first, but was too time consuming. The worst attempts were then found with the following code and the hyperparameters corresponding to those results were removed.

In [78]:
# print_best_params(grid_search)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results.sort_values(by='mean_test_score',ascending=True).head(5) # worst 5

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_mlp__activation,param_mlp__alpha,param_mlp__batch_size,param_mlp__hidden_layer_sizes,param_mlp__learning_rate,param_mlp__learning_rate_init,param_mlp__max_iter,param_mlp__power_t,param_mlp__random_state,param_mlp__shuffle,param_mlp__solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
672,0.073005,0.003535,0.001895,0.000299,relu,0,auto,"(18, 39)",invscaling,0.001,50,0.5,13,True,sgd,"{'mlp__activation': 'relu', 'mlp__alpha': 0, '...",0.277507,0.235556,0.439130,0.268622,0.499716,0.353846,0.305983,0.452070,0.463177,0.412051,0.370766,0.089472,862
528,0.082080,0.016156,0.002294,0.000457,relu,0.01,auto,"(18, 39)",invscaling,0.001,50,0.5,13,True,sgd,"{'mlp__activation': 'relu', 'mlp__alpha': 0.01...",0.277507,0.235556,0.439130,0.268622,0.499716,0.353846,0.305983,0.452070,0.463177,0.412051,0.370766,0.089472,862
816,0.075498,0.005353,0.003092,0.003040,relu,2,auto,"(18, 39)",invscaling,0.001,50,0.5,13,True,sgd,"{'mlp__activation': 'relu', 'mlp__alpha': 2, '...",0.277507,0.235556,0.439130,0.268622,0.499716,0.353846,0.305983,0.452070,0.463177,0.412051,0.370766,0.089472,862
384,0.068517,0.001548,0.002294,0.000639,tanh,2,auto,"(18, 39)",invscaling,0.001,50,0.5,13,True,sgd,"{'mlp__activation': 'tanh', 'mlp__alpha': 2, '...",0.252174,0.301401,0.397759,0.221719,0.427273,0.475936,0.299597,0.427273,0.481481,0.447022,0.373164,0.090745,859
96,0.089713,0.009330,0.002195,0.001075,tanh,0.01,auto,"(18, 39)",invscaling,0.001,50,0.5,13,True,sgd,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.01...",0.252174,0.301401,0.397759,0.221719,0.427273,0.475936,0.299597,0.427273,0.481481,0.447022,0.373164,0.090745,859


<a id='16'></a>
## 16) Summary

* Below are the tables of the specific feature selection method.
* The performance of the algorithms is in descending order.
* All the results are the average values of a 10-fold cross validation.
* The columns contain the accuracy and the average values of precision, recall and f1 score.
* It is observed that the number of samples of Βenign and Μalignant cancer are equal (212 respectively), so the weighted average and the macro average are equal.

<table>
    <tr>
        <th colspan="5"> K-Best : Default algorithms</th>
    </tr>
    <tr>
        <th></th>
        <th>precision </th>
        <th>recall</th>
        <th>f1 score</th>
        <th>accuracy</th>  
    </tr>
    <tr>
        <th>SVC</th>
        <td>0.972</td>
        <td>0.972</td>
        <td>0.972</td>
        <td>0.972</td>
    </tr>
    <tr>
        <th>Random Forest</th>
        <td>0.969</td>
        <td>0.969</td>
        <td>0.969</td>
        <td>0.969</td>
    </tr>
    <tr>
        <th>MLP</th>
        <td>0.967</td>
        <td>0.967</td>
        <td>0.967</td>
        <td>0.967</td>
    </tr>
    <tr>
        <th>AdaBoost</th>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.965</td>
    </tr>
    <tr>
        <th>KNN</th>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
    </tr>
    <tr>
        <th>SGD</th>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
    </tr>
    <tr>
        <th>LGBM</th>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
    </tr>
    <tr>
        <th>XGBoost</th>
        <td>0.958</td>
        <td>0.958</td>
        <td>0.958</td>
        <td>0.958</td>
    </tr>
    <tr>
        <th>Ridge</th>
        <td>0.957</td>
        <td>0.955</td>
        <td>0.955</td>
        <td>0.955</td>
    </tr>
    <tr>
        <th>LDA</th>
        <td>0.957</td>
        <td>0.955</td>
        <td>0.955</td>
        <td>0.955</td>
    </tr>
    <tr>
        <th>QDA</th>
        <td>0.946</td>
        <td>0.946</td>
        <td>0.946</td>
        <td>0.946</td>
    </tr>
    <tr>
        <th>GNB</th>
        <td>0.930</td>
        <td>0.929</td>
        <td>0.929</td>
        <td>0.929</td>
    </tr>
    <tr>
        <th>Decision Tree</th>
        <td>0.929</td>
        <td>0.929</td>
        <td>0.929</td>
        <td>0.929</td>
    </tr>

</table>

<table>
    <tr>
        <th colspan="5"> K-Best : Tuned algorithms</th>
    </tr>
    <tr>
        <th></th>
        <th>precision </th>
        <th>recall</th>
        <th>f1 score</th>
        <th>accuracy</th>  
    </tr>
    <tr>
        <th>MLP</th>
        <td>0.976</td>
        <td>0.976</td>
        <td>0.976</td>
        <td>0.976</td>
    </tr>
    <tr>
        <th>SGD</th>
        <td>0.974</td>
        <td>0.974</td>
        <td>0.974</td>
        <td>0.974</td>
    </tr>
    <tr>
        <th>SVC</th>
        <td>0.972</td>
        <td>0.972</td>
        <td>0.972</td>
        <td>0.972</td>
    </tr>
    <tr>
        <th>Random Forest</th>
        <td>0.972</td>
        <td>0.972</td>
        <td>0.972</td>
        <td>0.972</td>
    </tr>
    <tr>
        <th>AdaBoost</th>
        <td>0.972</td>
        <td>0.972</td>
        <td>0.972</td>
        <td>0.972</td>
    </tr>
    <tr>
        <th>LGBM</th>
        <td>0.970</td>
        <td>0.969</td>
        <td>0.969</td>
        <td>0.969</td>
    </tr>
    <tr>
        <th>XGBoost</th>
        <td>0.967</td>
        <td>0.967</td>
        <td>0.967</td>
        <td>0.967</td>
    </tr>
    <tr>
        <th>KNN</th>
        <td>0.962</td>
        <td>0.962</td>
        <td>0.962</td>
        <td>0.962</td>
    </tr>
    <tr>
        <th>Ridge</th>
        <td>0.962</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
    </tr>
    <tr>
        <th>LDA</th>
        <td>0.957</td>
        <td>0.955</td>
        <td>0.955</td>
        <td>0.955</td>
    </tr>
    <tr>
        <th>QDA</th>
        <td>0.946</td>
        <td>0.946</td>
        <td>0.946</td>
        <td>0.946</td>
    </tr>
    <tr>
        <th>Decision Tree</th>
        <td>0.943</td>
        <td>0.943</td>
        <td>0.943</td>
        <td>0.943</td>
    </tr>
    <tr>
        <th>GNB</th>
        <td>0.933</td>
        <td>0.932</td>
        <td>0.932</td>
        <td>0.932</td>
    </tr>

</table>